In [1]:
from random import randint

import numpy as np
import torch
import json
from collections import Counter
from tqdm import tqdm
import nltk
import regex as re
import string
from nltk.corpus import stopwords
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import os
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import preprocessing 
from sklearn.linear_model import LogisticRegression

# Get data - pre-processing

In [32]:
def word_tokenize(tokens):
    return [token.replace("''", '"').replace("``", '"') for token in nltk.word_tokenize(tokens)]


def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc((s))))




def get_2d_spans(text, tokenss):
    spanss = []
    cur_idx = 0
    for tokens in tokenss:
        spans = []
        for token in tokens:
            if text.find(token, cur_idx) < 0:
                print(tokens)
                print("{} {} {}".format(token, cur_idx, text))
                raise Exception()
            cur_idx = text.find(token, cur_idx)
            spans.append((cur_idx, cur_idx + len(token)))
            cur_idx += len(token)
        spanss.append(spans)
    return spanss


def get_word_span(context, wordss, start, stop):
    spanss = get_2d_spans(context, wordss)
    idxs = []
    for sent_idx, spans in enumerate(spanss):
        for word_idx, span in enumerate(spans):
            if not (stop <= span[0] or start >= span[1]):
                idxs.append((sent_idx, word_idx))
    #print(spanss)
    #print(start,stop)
    #print(context[start:stop])
    assert len(idxs) > 0, "{} {} {} {}".format(context, spanss, start, stop)
    return idxs[0], (idxs[-1][0], idxs[-1][1] + 1)
def get_word_idx(context, wordss, idx):
    spanss = get_2d_spans(context, wordss)
    return spanss[idx[0]][idx[1]][0]



def process_tokens1(tokens):
   
    
    tokens = [w for w in tokens if w not in set(string.punctuation)]
   # stop_words = set(stopwords.words('english'))
    #tokens = [w for w in tokens if not w in stop_words]
    
    return tokens



def process_tokens(temp_tokens):
    tokens = []
    
    for token in temp_tokens:
        flag = False
        l = ("-", "\u2212", "\u2014", "\u2013", "/", "~", '"', "'", "\u201C", "\u2019", "\u201D", "\u2018", "\u00B0")
        # \u2013 is en-dash. Used for number to nubmer
        # l = ("-", "\u2212", "\u2014", "\u2013")
        # l = ("\u2013",)
        tokens.extend(re.split("([{}])".format("".join(l)), token))
    
    return tokens

def generatedataset(sentence_list,question_list):
    s_list=[]
    q_list=[]
    a_list=[]
    q_id_list=[]
     
    res=[]
    #i=0
    for q in question_list:
        #i+=1
        if(len(q)==0):
            #print(i)
            continue
        ai=q[0][0]
        pi=q[0][1]
        si=q[0][2]
        
        for i, sent in enumerate(sentence_list[ai][pi]):
            
            s_list+=[' '.join(sent)]
            q_list+=[' '.join(q[0][3])]
            q_id_list+=[q[0][5]]
            if i == si:
                a_list+=[q[0][4]]
                res+=[[q[0][3],sent,q[0][4],q[0][5]]]
            else:
                a_list+=[0]
                res+=[[q[0][3],sent,0,q[0][5]]]
            
        
#         res+=[[q[0][3],sentence_list[ai][pi][si],q[0][4], q[0][5]]]
#         s_list+=[' '.join(sentence_list[ai][pi][si])]
#         q_list+=[' '.join(q[0][3])]
#         a_list+=[q[0][4]]
        #for i in range(len(sentence_list[ai][pi])):
            #if(i==si):
                #res+=[[q[0][3],sentence_list[ai][pi][i],q[0][4], q[0][5]]]
            #else:
            #    res+=[[q[0][3],sentence_list[ai][pi][i],0, q[0][5]]]
            
    return res, s_list, q_list, a_list, q_id_list


def create_index_dict(embed,word_counter):
    word2idx={}
    idx2word={}
    i=1
    word2idx['<UNK>']=0
    idx2word[0]='<UNK>'
    vocab=np.array(['<UNK>'])
    for (key,value) in (embed.items()):
        word2idx[key]=i
        idx2word[i]=key
   #     vocab=np.append(vocab,[key])
        i+=1
    
    
    for key,value in (word_counter.items()):
        key=key.lower()
    
        try:
            embed[key]
           
        except: 
            try:
                word2idx[key]
            except:
                word2idx[key]=i
                idx2word[i]=key
        #        vocab=np.append(vocab,[key])
                i+=1
   
   # assert len(word2idx.keys())==len(vocab)
    return word2idx,idx2word,set(word2idx.keys())

def process(data):
    q_trn=[]
    s_trn=[]
    y_trn=[]
    q_id=[]
    for i in range(len(data)):
       # tokens = [w for w in tokens if w not in set(string.punctuation) ]
        
        
        question=list(filter(None, data[i][0]))#filter empty strin
        sentence=list(filter(None, data[i][1]))#filter empty strin
        index=data[i][2]
       # list1+=[[question,sentence,index]]
        
        q_trn+=[question]
        s_trn+=[sentence]
        y_trn+=[index]
        q_id+=[data[i][3]]
    return q_trn,s_trn,y_trn, q_id

In [3]:
a=[1,2,3]
a.append([4,5,6])
a

[1, 2, 3, [4, 5, 6]]

In [4]:
data_type='training'
source_path = "{}.json".format(data_type)
source_data = json.load(open(source_path, 'r'))

data_type='training'
split=True
nltk.data.path.append("/afs/cs.stanford.edu/u/tianzhao/tz/nltk_data")
sent_tokenize = nltk.sent_tokenize

In [33]:
nltk.data.path.append("/afs/cs.stanford.edu/u/tianzhao/tz/nltk_data")
sent_tokenize = nltk.sent_tokenize
word_counter = Counter()


In [6]:
context_list=[]
sentence_list=[]
question_list=[]
answer_list=[]
answer_index=[]
label_index=[]
label=[]
idxs=[]


word_counter = Counter()

for ai, article in enumerate(tqdm(source_data['data'])):
    s=[]
    p = []
    for pi, para in enumerate(article['paragraphs']):
        # wordss
        context = para['context']
        context = context.replace("''", '" ')
        context = context.replace("``", '" ')
       
        si = list(map(word_tokenize, sent_tokenize(context)))
       # print(si)
        si = [process_tokens(tokens) for tokens in si]  # process tokens
        for sentence in si:
            for word in sentence:
                 word_counter[word] += 1
 
        # given xi, add chars
        s.append(si)
        p.append(context)

        indexi = [ai, pi]
        for qa in para['qas']:
                qi = word_tokenize(qa['question'])
                if qa['is_impossible']:
                    labeli=[0]
                    #print(labeli)
                else:
                    labeli=[1]       
                yi = []
                yyi=[] 
                answers = []
                q_id = qa['id']
                
                ans=[]
                if labeli[0] == 1:
                    ans=qa['answers']
                else:
                    ans=qa['plausible_answers']
                    
                for answer in ans:
                    answer_text=answer['text']
                    answers.append(answer_text)
                    answer_start = answer['answer_start']
                    answer_stop = answer_start + len(answer_text)
                   # print(context)
                   # print(si)
                    yi0, yi1 = get_word_span(context, si, answer_start, answer_stop)
                    #w0 = xi[yi0[0]][yi0[1]]            
                    #w1 = xi[yi1[0]][yi1[1]-1]
                    yi.append([ai,pi,yi0[0],qi,labeli[0],q_id]) 
                    #if labeli[0] == 0:
                        #print(yi)
                   # print(yi)
                   # print(yi)
                    yyi.append([answer_start,answer_stop])
                for qij in qi:            
                         word_counter[qij] += 1
                        
                        
                        
                question_list.append(qi)
                answer_list.append(answers)
                answer_index.append(yi)
                label.append(labeli)
                label_index.append(indexi)
               # ids.append(qa['id'])
                idxs.append(len(idxs))
         
    sentence_list.append(s)

    context_list.append(p)

100%|██████████| 442/442 [00:47<00:00,  8.80it/s]


In [7]:
X, s_list, q_list, a_list, q_id_list =generatedataset(sentence_list,answer_index)   

In [9]:
ctr=0
for l in answer_index:
    try:
        if l[0][4] ==0:
            ctr+=1
    except:
        continue
ctr        

34798

In [10]:
print(len(X))
print(len(s_list))
print(len(q_list))
print(len(a_list))

352635
352635
352635
352635


In [13]:
s_list[:20]

['Beyoncé Giselle Knowles - Carter (  / biːˈjɒnseɪ /  bee - YON - say ) ( born September 4 , 1981 ) is an American singer , songwriter , record producer and actress .',
 "Born and raised in Houston , Texas , she performed in various singing and dancing competitions as a child , and rose to fame in the late 1990s as lead singer of R & B girl - group Destiny  ' s Child .",
 "Managed by her father , Mathew Knowles , the group became one of the world  ' s best - selling girl groups of all time .",
 'Their hiatus saw the release of Beyoncé  \' s debut album , Dangerously in Love ( 2003 ) , which established her as a solo artist worldwide , earned five Grammy Awards and featured the Billboard Hot 100 number - one singles  "  Crazy in Love  "  and  "  Baby Boy  "  .',
 'Beyoncé Giselle Knowles - Carter (  / biːˈjɒnseɪ /  bee - YON - say ) ( born September 4 , 1981 ) is an American singer , songwriter , record producer and actress .',
 "Born and raised in Houston , Texas , she performed in var

# Using pre-trained InferSent

In [90]:
# Load model
from models import InferSent
model_version = 1
MODEL_PATH = "./encoder/infersent%s.pkl" % model_version
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))


# Keep it on CPU or put it on GPU
use_cuda = False
model = model.cuda() if use_cuda else model


# If infersent1 -> use GloVe embeddings. If infersent2 -> use InferSent embeddings.
W2V_PATH = 'glove.840B.300d.txt' if model_version == 1 else '../dataset/fastText/crawl-300d-2M.vec'
model.set_w2v_path(W2V_PATH)

# Load embeddings of K most frequent words
model.build_vocab_k_words(K=1000000)

Vocab size : 1000000


# Load and encode sentences

In [92]:
def get_sent_embedding(sent_list):
    embeddings = model.encode(sent_list, bsize=128, tokenize=False, verbose=True)
    return embeddings

    

In [16]:
q_embed = get_sent_embedding(q_list)
s_embed = get_sent_embedding(s_list)

Nb words kept : 4550040/4582850 (99.3%)
Speed : 146.6 sentences/s (cpu mode, bsize=128)
Nb words kept : 10492107/10573990 (99.2%)
Speed : 60.1 sentences/s (cpu mode, bsize=128)


In [17]:
np.save('q_embed_2', q_embed)
np.save('s_embed_2', s_embed)


In [18]:
np.save('a_list_2', a_list)

In [8]:
q_embed = np.load('q_embed_2.npy')
s_embed = np.load('s_embed_2.npy')
a_list = np.load('a_list_2.npy')

In [5]:
q_embed.shape

(352635, 4096)

In [6]:
s_embed.shape

(352635, 4096)

# Cosine Similarity - Semantic Similarity

In [9]:
def cos_sim_loop(q_embed, s_embed):
    sim = []
    for i in range(q_embed.shape[0]):
        prod = q_embed[i,:].dot(s_embed[i,:])
        prod /= np.sqrt((q_embed[i,:] ** 2).sum())
        prod /= np.sqrt((s_embed[i,:] ** 2).sum())
        sim+=[prod]
    return sim

In [10]:
cos_sim = cos_sim_loop(q_embed, s_embed)

In [33]:
cos_sim[0]

-0.08217849

In [11]:
#get sentence based on max sim
q_id_max = {}
for i in range(len(q_id_list)):
    #find max sim score for q_id
      
    if q_id_list[i] in q_id_max.keys():
        if cos_sim[i] > cos_sim[q_id_max[q_id_list[i]]]:
            q_id_max[q_id_list[i]] = i
    else:
        q_id_max[q_id_list[i]] = i
    
    
    
    
    


In [12]:
ind2keep = sorted(list(q_id_max.values()))
new_q_id_list = [q_id_list[i] for i in ind2keep]
new_q_list = [q_list[i] for i in ind2keep]
new_a_list = [a_list[i] for i in ind2keep]
new_s_list = [s_list[i] for i in ind2keep]

new_q_embed =  q_embed[ind2keep[:],:]
new_s_embed =  s_embed[ind2keep[:],:]

In [9]:
def cosineimilarity(e,v):
  
    #Input:
    #e = nxd input matrix with n row-vectors of dimensionality d (n is number of dictionary_keys)
    #v = mxd input matrix with m row-vectors of dimensionality d (m is number of test samples)
    # Output:
    # Matrix D of size nxm
    # s(i,j) is the cosinesimiarlity of embed(i,:) and test(j,:)
    g=e.dot(v.T)
    b=np.expand_dims(np.linalg.norm(e,axis=1),1)
    a=np.expand_dims(np.linalg.norm(v,axis=1),1)
    s=np.divide(g,np.multiply(b,a.T))
    # ... until here
    return s




In [ ]:
sim_mat = cosineimilarity(q_embed,s_embed)

In [13]:
def get_pred_cos_sim(sim_mat, threshold=0.6):
    preds=[]
    
    sim = sim_mat.diagonal()
    
    for i in range(sim.shape[0]):
        
        if abs(sim[i])>threshold:
            preds+=[1]
        else:
            preds+=[0]
    return sim,preds

def get_acc(preds,gold_labels):
    correct=0;
    total=len(preds)
    for i in range(total):
        if preds[i]==gold_labels[i]:
            correct+=1
    return correct/total

In [ ]:
cos_sim, preds = get_pred_cos_sim(sim_mat)


# Syntactic Similarity

In [14]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem.snowball import SnowballStemmer

In [15]:
stop_words = set(stopwords.words('english')) 
stemmer = SnowballStemmer("english")

def preprocess_syntactic(sent_list):
    result=[]
    for sent in sent_list:
        filtered_sentence = []
        word_tokens = word_tokenize(sent) 
        filtered_sentence = [stemmer.stem(w) for w in word_tokens if not w in stop_words] 
        result+=[filtered_sentence]
    return result

In [16]:
q_list_syn = preprocess_syntactic(q_list)
s_list_syn = preprocess_syntactic(s_list)

In [17]:
def get_syntactic_non_similarity_score(s_list_syn, q_list_syn):
    
    result=[]
    
    for i,q in enumerate(q_list_syn):
         
        ctr = 0
        
        for w in q:
            if w in list(set(s_list_syn[i])):
                ctr+=1
        result+=[ ctr / len(q) if len(q)>0 else 0]
        
    return result

In [18]:
syn_sim = get_syntactic_non_similarity_score(s_list_syn, q_list_syn)

In [67]:
syn_sim

[0.125,
 0.0,
 0.0,
 0.625,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.42857142857142855,
 0.2857142857142857,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.1,
 0.0,
 0.0,
 0.5,
 0.375,
 0.125,
 0.125,
 0.25,
 0.25,
 0.1111111111111111,
 0.1111111111111111,
 0.4444444444444444,
 0.1111111111111111,
 0.1111111111111111,
 0.1111111111111111,
 0.0,
 0.0,
 0.0,
 0.0,
 0.1111111111111111,
 0.0,
 0.5555555555555556,
 0.0,
 0.0,
 0.2,
 0.0,
 0.0,
 0.1,
 0.5,
 0.0,
 0.16666666666666666,
 0.0,
 0.0,
 0.3333333333333333,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.125,
 0.25,
 0.0,
 0.0,
 0.125,
 0.375,
 0.125,
 0.2857142857142857,
 0.42857142857142855,
 0.42857142857142855,
 0.0,
 0.42857142857142855,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.4,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5,
 0.0,
 0.0,
 0.0,
 0.25,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.2,
 0.0,
 0.0,
 0.0,
 0.0,
 0.375,
 0.0,
 0.25,
 0.375,
 0.125,
 0.125,
 0.42857142857142855,
 0.0,
 0.2857142857142857,
 0.5714285714285714,
 0.2857142857142857,
 0.0,
 0.

In [19]:
#get sentence based on max syn sim
q_id_max_syn = {}
for i in range(len(q_id_list)):
    #find max sim score for q_id
      
    if q_id_list[i] in q_id_max_syn.keys():
        if syn_sim[i] > syn_sim[q_id_max_syn[q_id_list[i]]]:
            q_id_max_syn[q_id_list[i]] = i
    else:
        q_id_max_syn[q_id_list[i]] = i
  

In [20]:
ind2keep_syn = sorted(list(q_id_max_syn.values()))
new_q_id_list_syn = [q_id_list[i] for i in ind2keep_syn]
new_q_list_syn = [q_list[i] for i in ind2keep_syn]
new_a_list_syn = [a_list[i] for i in ind2keep_syn]
new_s_list_syn = [s_list[i] for i in ind2keep_syn]

new_q_embed_syn =  q_embed[ind2keep_syn[:],:]
new_s_embed_syn =  s_embed[ind2keep_syn[:],:]

In [21]:
new_cos_sim = [cos_sim[i] for i in ind2keep]
new_syn_sim = [syn_sim[i] for i in ind2keep_syn]

In [22]:
print(len(new_syn_sim))
print(len(new_syn_sim))

69596
69596


In [23]:
new_cos_sim = np.array(new_cos_sim).reshape(-1,1)
new_syn_sim = np.array(new_syn_sim).reshape(-1,1)

# LR

In [24]:
xTr = np.concatenate([new_q_embed, new_s_embed , new_s_embed_syn, new_cos_sim, new_syn_sim], axis=1)
yTr = max(new_a_list_syn, new_a_list)

In [25]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression


In [41]:
clf = LogisticRegression(random_state=0, solver='lbfgs').fit(xTr, yTr)

# Dev Set

In [34]:
te_data_type='development'
te_source_path = "{}.json".format(te_data_type)
te_source_data = json.load(open(te_source_path, 'r'))



te_context_list=[]
te_sentence_list=[]
te_question_list=[]
te_answer_list=[]
te_answer_index=[]
te_label_index=[]
te_label=[]
te_idxs=[]
te_word_counter = Counter()

for ai, article in enumerate(tqdm(te_source_data['data'])):
    s=[]
    p = []
    for pi, para in enumerate(article['paragraphs']):
        # wordss
        context = para['context']
        context = context.replace("''", '" ')
        context = context.replace("``", '" ')
       
        si = list(map(word_tokenize, sent_tokenize(context)))
       # print(si)
        si = [process_tokens(tokens) for tokens in si]  # process tokens
        for sentence in si:
            for word in sentence:
                 word_counter[word] += 1
 
        # given xi, add chars
        s.append(si)
        p.append(context)

        indexi = [ai, pi]
        for qa in para['qas']:
                qi = word_tokenize(qa['question'])
                
                if qa['is_impossible']:
                    labeli=[0]
                  #  print(labeli)
                else:
                    labeli=[1] 
                       
                yi = []
                yyi=[] 
                answers = []
                q_id = qa['id']
                
                ans=[]
                if labeli[0] == 1:
                    ans=qa['answers']
                else:
                    ans=qa['plausible_answers']
                    
                
                for answer in ans:
                    answer_text=answer['text']
                    answers.append(answer_text)
                    answer_start = answer['answer_start']
                    answer_stop = answer_start + len(answer_text)
                   # print(context)
                   # print(si)
                    yi0, yi1 = get_word_span(context, si, answer_start, answer_stop)
                    #w0 = xi[yi0[0]][yi0[1]]            
                    #w1 = xi[yi1[0]][yi1[1]-1]
                    yi.append([ai,pi,yi0[0],qi,labeli[0],q_id]) 
                   # print(yi)
                   # print(yi)
                    yyi.append([answer_start,answer_stop])
                for qij in qi:
                    
                            word_counter[qij] += 1
                te_question_list.append(qi)
                te_answer_list.append(answers)
                te_answer_index.append(yi)
                te_label.append(labeli)
                te_label_index.append(indexi)
               # ids.append(qa['id'])
                te_idxs.append(len(idxs))
         
    te_sentence_list.append(s)

    te_context_list.append(p)

100%|██████████| 442/442 [00:39<00:00, 11.29it/s]


In [35]:
te_X, te_s_list, te_q_list, te_a_list, te_q_id_list=generatedataset(te_sentence_list,te_answer_index)   

In [99]:
len(set(te_s_list))

53971

In [25]:
q_id

'5a7e070b70df9f001a87543a'

In [16]:
print(len(te_X))
print(len(te_s_list))
print(len(te_q_list))
print(len(te_a_list))

17400
17400
17400
17400


In [17]:
Counter(te_a_list)

Counter({0: 8700, 1: 8700})

In [93]:
te_q_embed = get_sent_embedding(te_q_list)
te_s_embed = get_sent_embedding(te_s_list)


Nb words kept : 1140849/1149398 (99.3%)
Speed : 141.2 sentences/s (cpu mode, bsize=128)
Nb words kept : 2625053/2645250 (99.2%)
Speed : 64.5 sentences/s (cpu mode, bsize=128)


In [101]:
np.save('te_q_embed_2',te_q_embed)
np.save('te_s_embed_2',te_s_embed)

In [28]:
te_q_embed = np.load('te_q_embed_2.npy')
te_s_embed = np.load('te_s_embed_2.npy')

In [102]:
te_s_embed.shape

(88248, 4096)

In [36]:
te_cos_sim = cos_sim_loop(te_q_embed, te_s_embed)
#get sentence based on max sim
te_q_id_max = {}
for i in range(len(te_q_id_list)):
    #find max sim score for q_id
      
    if te_q_id_list[i] in te_q_id_max.keys():
        if te_cos_sim[i] > te_cos_sim[te_q_id_max[te_q_id_list[i]]]:
            te_q_id_max[te_q_id_list[i]] = i
    else:
        te_q_id_max[te_q_id_list[i]] = i
    
    
    
    
te_ind2keep = sorted(list(te_q_id_max.values()))
te_new_q_id_list = [te_q_id_list[i] for i in te_ind2keep]
te_new_q_list = [te_q_list[i] for i in te_ind2keep]
te_new_a_list = [te_a_list[i] for i in te_ind2keep]
te_new_s_list = [te_s_list[i] for i in te_ind2keep]

te_new_q_embed =  te_q_embed[te_ind2keep[:],:]
te_new_s_embed =  te_s_embed[te_ind2keep[:],:]


In [37]:

te_q_list_syn = preprocess_syntactic(te_q_list)
te_s_list_syn = preprocess_syntactic(te_s_list)


te_syn_sim = get_syntactic_non_similarity_score(te_s_list_syn, te_q_list_syn)

#get sentence based on max syn sim
te_q_id_max_syn = {}
for i in range(len(te_q_id_list)):
    #find max sim score for q_id
      
    if te_q_id_list[i] in te_q_id_max_syn.keys():
        if te_syn_sim[i] > te_syn_sim[te_q_id_max_syn[te_q_id_list[i]]]:
            te_q_id_max_syn[te_q_id_list[i]] = i
    else:
        te_q_id_max_syn[te_q_id_list[i]] = i
  

te_ind2keep_syn = sorted(list(te_q_id_max_syn.values()))
te_new_q_id_list_syn = [te_q_id_list[i] for i in te_ind2keep_syn]
te_new_q_list_syn = [te_q_list[i] for i in te_ind2keep_syn]
te_new_a_list_syn = [te_a_list[i] for i in te_ind2keep_syn]
te_new_s_list_syn = [te_s_list[i] for i in te_ind2keep_syn]

te_new_q_embed_syn =  te_q_embed[te_ind2keep_syn[:],:]
te_new_s_embed_syn =  te_s_embed[te_ind2keep_syn[:],:]

In [38]:
te_new_cos_sim = [te_cos_sim[i] for i in te_ind2keep]
te_new_syn_sim = [te_syn_sim[i] for i in te_ind2keep_syn]

te_new_cos_sim = np.array(te_new_cos_sim).reshape(-1,1)
te_new_syn_sim = np.array(te_new_syn_sim).reshape(-1,1)

In [39]:
xTe = np.concatenate([te_new_q_embed, te_new_s_embed , te_new_s_embed_syn, te_new_cos_sim, te_new_syn_sim], axis=1)


In [42]:
yTe = clf.predict(xTe)

In [67]:
get_acc(yTr,a_list)

0.5753204207138342

In [75]:
pred_dict = get_prediction_dict(yTe, te_new_q_id_list)

In [78]:
write_result('model2-dev.csv',pred_dict )

In [81]:
set(pred_dict.values())

{0, 1}

In [84]:
def calculate_precision(true_positives, false_positives):
    if true_positives + false_positives == 0:
        return 0
    return true_positives / (true_positives + false_positives)


def calculate_recall(true_positives, false_negatives):
    if true_positives + false_negatives == 0:
        return 0
    return true_positives / (true_positives + false_negatives)


def calculate_f1(precision, recall):
    if precision + recall == 0:
        return 0
    return (2 * precision * recall) / (precision + recall)


def calculate_accuracy(true_positive, true_negative, false_positive, false_negative):
    return (true_positive + true_negative) / (true_positive + true_negative + false_positive + false_negative)


In [98]:


def evaluate(pred, gold_labels):
    """

    :param preds: A dictionary mapping question IDs to:
                    -> 0 if the question does not have a plausible answer
                    -> 1 if the question does have a plausible answer

    :param test_set: A dictionary of the following form:

    test_set["data"] -> list of titles
        title["title"] -> title name
        title["paragraphs"] -> list of paragraphs
            paragraph["context"] -> data which might answer questions in this paragraph
            paragraph["qas"] -> list of questions
                question["question"] -> question content
                question["id"] -> question ID

    :return: A 3-tuple consisting of (precision, recall, f1_score)
    """

#     correct_mapping = {}

    true_positives = 0
    true_negatives = 0
    false_positives = 0
    false_negatives = 0

#     for title in test_set["data"]:
#         for paragraph in title["paragraphs"]:
#             for question in paragraph["qas"]:
#                 if question["is_impossible"]:
#                     correct_mapping[question["id"]] = 0
#                 else:
#                     correct_mapping[question["id"]] = 1

#     for id, pred in correct_mapping.items():
#         if pred == 0 and preds[id] == 0:
#             true_negatives += 1
#         elif pred == 1 and preds[id] == 1:
#             true_positives += 1
#         elif pred == 0 and preds[id] == 1:
#             false_positives += 1
#         elif pred == 1 and preds[id] == 0:
#             false_negatives += 1
     
    if len(pred) != len(gold_labels):
        print("Error")
        return
            
    for i in range(len(pred)):
        if pred[i] == 0 and gold_labels[i] == 0:
            true_negatives += 1
        elif pred[i] == 1 and gold_labels[i] == 1:
            true_positives += 1
        elif pred[i] == 0 and gold_labels[i] == 1:
            false_positives += 1
        elif pred[i] == 1 and gold_labels[i] == 0:
            false_negatives += 1

    print("Intermediate evaluation:")
    print("true_positives: " + str(true_positives))
    print("true_negatives: " + str(true_negatives))
    print("false_positives: " + str(false_positives))
    print("false_negatives: " + str(false_negatives))
    print("\n" * 3)

    precision = calculate_precision(true_positives, false_positives)
    recall = calculate_recall(true_positives, false_negatives)
    f_1 = calculate_f1(precision, recall)
    accuracy = calculate_accuracy(true_positives, true_negatives, false_positives, false_negatives)
    return precision, recall, f_1, accuracy




In [94]:
def evaluate(pred, test_set):
    """

    :param preds: A dictionary mapping question IDs to:
                    -> 0 if the question does not have a plausible answer
                    -> 1 if the question does have a plausible answer

    :param test_set: A dictionary of the following form:

    test_set["data"] -> list of titles
        title["title"] -> title name
        title["paragraphs"] -> list of paragraphs
            paragraph["context"] -> data which might answer questions in this paragraph
            paragraph["qas"] -> list of questions
                question["question"] -> question content
                question["id"] -> question ID

    :return: A 3-tuple consisting of (precision, recall, f1_score)
    """
    
    true_positives = 0
    true_negatives = 0
    false_positives = 0
    false_negatives = 0
    n=len(pred)
    for i in range(n):
        if pred[i] == 0 and test_set[i] == 0:
            true_negatives += 1
        elif pred[i] == 1 and test_set[i]  == 1:
            true_positives += 1
        elif pred[i]== 0 and test_set[i] == 1:
            false_positives += 1
        elif pred[i] == 1 and test_set[i] == 0:
            false_negatives += 1

    precision = calculate_precision(true_positives, false_positives)
    recall = calculate_recall(true_positives, false_negatives)
    f_1 = calculate_f1(precision, recall)
    acc=(true_positives+true_negatives)/(true_positives+true_negatives+false_positives+false_negatives)
    return precision, recall, f_1,acc

In [99]:
evaluate(yTe, te_new_a_list )

Intermediate evaluation:
true_positives: 65
true_negatives: 15078
false_positives: 2101
false_negatives: 156






(0.030009233610341645,
 0.29411764705882354,
 0.054461667364893177,
 0.8702873563218391)

In [96]:
len(yTe)

17400

In [97]:
len(te_new_a_list)

17400

# Test Set

In [49]:
te_data_type='test'
te_source_path = "{}.json".format(te_data_type)
te_source_data = json.load(open(te_source_path, 'r'))

test_q=[]
test_q_id=[]
test_sen=[]

for ai, article in enumerate(tqdm(te_source_data['data'])):
    s=[]
    p = []
    for pi, para in enumerate(article['paragraphs']):
        # wordss
        context = para['context']
        context = context.replace("''", '" ')
        context = context.replace("``", '" ')
       
        si = list(map(word_tokenize, sent_tokenize(context)))
       # print(si)
        si = [process_tokens(tokens) for tokens in si]  # process tokens
        for sentence in si:
            for word in sentence:
                 word_counter[word] += 1
 


            for qa in para['qas']:
                    qi = word_tokenize(qa['question'])
                    q_id = qa['id']

                    test_q.append(' '.join(qi))
                    test_q_id.append(q_id)
                    test_sen.append(' '.join(sentence))


               
         


100%|██████████| 35/35 [00:09<00:00,  3.54it/s]


In [50]:
test_q_id_list = test_q_id
test_s_list =  test_sen
test_q_list = test_q

In [34]:
test_q_embed = get_sent_embedding(test_q)
test_s_embed = get_sent_embedding(test_sen)


Nb words kept : 836633/843857 (99.1%)
Speed : 144.9 sentences/s (cpu mode, bsize=128)
Nb words kept : 1890676/1907098 (99.1%)
Speed : 67.8 sentences/s (cpu mode, bsize=128)


In [35]:
np.save('test_q_embed', test_q_embed)
np.save('test_s_embed', test_s_embed)
np.save('test_q_id',test_q_id)

np.save('te_q_embed', te_q_embed)
np.save('te_s_embed', te_s_embed)


In [36]:
np.save('te_a_list', te_a_list)


In [53]:
test_q_embed = np.load('test_q_embed.npy')
test_s_embed = np.load('test_s_embed.npy')
test_q_id_list = np.load('test_q_id.npy')

In [54]:
test_cos_sim = cos_sim_loop(test_q_embed, test_s_embed)
#get sentence based on max sim
test_q_id_max = {}
for i in range(len(test_q_id_list)):
    #find max sim score for q_id
      
    if test_q_id_list[i] in test_q_id_max.keys():
        if test_cos_sim[i] > test_cos_sim[test_q_id_max[test_q_id_list[i]]]:
            test_q_id_max[test_q_id_list[i]] = i
    else:
        test_q_id_max[test_q_id_list[i]] = i
    
    
    
    
test_ind2keep = sorted(list(test_q_id_max.values()))
test_new_q_id_list = [test_q_id_list[i] for i in test_ind2keep]
# test_new_q_list = [test_q_list[i] for i in test_ind2keep]
# test_new_a_list = [test_a_list[i] for i in test_ind2keep]
# test_new_s_list = [test_s_list[i] for i in test_ind2keep]

test_new_q_embed =  test_q_embed[test_ind2keep[:],:]
test_new_s_embed =  test_s_embed[test_ind2keep[:],:]


In [55]:

test_q_list_syn = preprocess_syntactic(test_q_list)
test_s_list_syn = preprocess_syntactic(test_s_list)


test_syn_sim = get_syntactic_non_similarity_score(test_s_list_syn, test_q_list_syn)

#get sentence based on max syn sim
test_q_id_max_syn = {}
for i in range(len(test_q_id_list)):
    #find max sim score for q_id
      
    if test_q_id_list[i] in test_q_id_max_syn.keys():
        if test_syn_sim[i] > test_syn_sim[test_q_id_max_syn[test_q_id_list[i]]]:
            test_q_id_max_syn[test_q_id_list[i]] = i
    else:
        test_q_id_max_syn[test_q_id_list[i]] = i
  

test_ind2keep_syn = sorted(list(test_q_id_max_syn.values()))
test_new_q_id_list_syn = [test_q_id_list[i] for i in test_ind2keep_syn]
# test_new_q_list_syn = [test_q_list[i] for i in test_ind2keep_syn]
# test_new_a_list_syn = [test_a_list[i] for i in test_ind2keep_syn]
# test_new_s_list_syn = [test_s_list[i] for i in test_ind2keep_syn]

test_new_q_embed_syn =  test_q_embed[test_ind2keep_syn[:],:]
test_new_s_embed_syn =  test_s_embed[test_ind2keep_syn[:],:]

In [56]:
test_new_cos_sim = [test_cos_sim[i] for i in test_ind2keep]
test_new_syn_sim = [test_syn_sim[i] for i in test_ind2keep_syn]

test_new_cos_sim = np.array(test_new_cos_sim).reshape(-1,1)
test_new_syn_sim = np.array(test_new_syn_sim).reshape(-1,1)

In [57]:
xTest = np.concatenate([test_new_q_embed, test_new_s_embed , test_new_s_embed_syn, test_new_cos_sim, test_new_syn_sim], axis=1)

In [58]:
yTest = clf.predict(xTest)

In [59]:
def get_prediction_dict(preds, q_ids):
    if(len(preds) != len(q_ids)):
        print("Error in pred len")
        return
    result={}
    for i in range(len(preds)):
        result[q_ids[i]] = preds[i]
        
    return result

In [60]:
import csv
def write_result(filename,preds):
    with open(filename, 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['Id','Category'])
        for k,v in preds.items():
            filewriter.writerow([k,v])

In [61]:
test_preds = get_prediction_dict(yTest, test_new_q_id_list)

In [62]:
write_result('lr2.csv',test_preds)

In [68]:
len(test_preds.keys())

11856

In [69]:
keys = test_preds.keys()

In [70]:
one_pred={}
for k in keys:
    one_pred[k]=1

In [71]:
write_result('one.csv',one_pred)

# Retraining

In [ ]:
xTrCombined = np.concatenate([xTr,xTe], axis=0)
yTrCombined = np.concatenate([yTr,yTe], axis=0)
clf = LogisticRegression(random_state=0, solver='lbfgs').fit(xTrCombined, yTrCombined)

In [ ]:
yTrCombined_pred = clf.predict(xTrCombined)

In [ ]:
get_acc(yTrCombined_pred, yTrCombined)